In [422]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy, resample_apply
from backtesting.test import SMA
from backtesting import Backtest

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd 
import math as m
import os, glob

In [423]:
import warnings
warnings.filterwarnings("ignore")

In [424]:
PRED = True
WithClassOne = False
CASH = 1000
COMMISSION = 0.0001

In [425]:
pwd = os.getcwd()
datasource_path = os.path.join(pwd, r'../../../Results/LSTM/')
filenames = glob.glob(datasource_path + "/*.csv")
filenames.sort()
for file in filenames:
    print(os.path.basename(file))

3C-1H-LSTM_1H_43F_Class_01_Test.csv
3C-1H-LSTM_1H_43F_Class_01_Train.csv
C-LSTM_1D_19F_Class_03_Test.csv
C-LSTM_1D_19F_Class_03_Train.csv
C-LSTM_1H_0F_Class_Test_Results_01.csv
C-LSTM_1H_0F_Class_Test_Results_02.csv
C-LSTM_1H_0F_Class_Train_Results_01.csv
C-LSTM_1H_0F_Class_Train_Results_02.csv


In [426]:
index = 0

dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
DATASET = pd.read_csv(filenames[index], header=0)
MODEL_NAME = os.path.basename(filenames[index])
print(MODEL_NAME)
print(DATASET.head(20))
print(DATASET.shape)


3C-1H-LSTM_1H_43F_Class_01_Test.csv
       Open     High      Low    Close      Volume  Real_Class  \
0   1.05809  1.05898  1.05769  1.05892   6150.9399         1.0   
1   1.05891  1.05986  1.05847  1.05958  11677.7197        -1.0   
2   1.05957  1.06041  1.05744  1.05765  15785.2197         1.0   
3   1.05765  1.05956  1.05708  1.05935  14816.4902         0.0   
4   1.05934  1.05946  1.05835  1.05927  10641.8301         1.0   
5   1.05925  1.06104  1.05890  1.06081  13180.9697         0.0   
6   1.06082  1.06181  1.06036  1.06089  14419.7803         1.0   
7   1.06092  1.06179  1.06055  1.06139  15307.1904        -1.0   
8   1.06136  1.06153  1.05658  1.05724  25968.0293        -1.0   
9   1.05719  1.05900  1.05668  1.05668  19795.0098         1.0   
10  1.05670  1.05841  1.05662  1.05801  16532.0801        -1.0   
11  1.05799  1.05970  1.05657  1.05687  11651.7695         0.0   
12  1.05688  1.05776  1.05687  1.05708   8330.3096        -1.0   
13  1.05707  1.05717  1.05573  1.05644  

In [427]:
if NumCLASS == 3:
    if (PRED):
        DATASET['Position'] = DATASET['Predicted_Class']
        DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = -1 
        DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 1 
    else:
        DATASET['Position'] = DATASET['Real_Class']
        DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = -1
        DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 1 
if NumCLASS == 5:
    if (PRED):
        DATASET['Position'] = DATASET['Predicted_Class']
        if (WithClassOne):
            DATASET.loc[DATASET.Predicted_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Predicted_Class == 2, 'Position'] = 1
        else:
            DATASET.loc[DATASET.Predicted_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Predicted_Class == 2, 'Position'] = 1
            DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = 0 # remove Small movements
            DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 0 #Remove Small movements
    else:
        DATASET['Position'] = DATASET['Real_Class']
        if (WithClassOne):
            DATASET.loc[DATASET.Real_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Real_Class == 2, 'Position'] = 1
        else:
            DATASET.loc[DATASET.Real_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Real_Class == 2, 'Position'] = 1
            DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = 0 # remove Small movements
            DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 0 #Remove Small movements
DATASET.head(10)

,Open,High,Low,Close,Volume,Real_Class,Predicted_Class,Position
0,1.05809,1.05898,1.05769,1.05892,6150.9399,1.0,0.0,0.0
1,1.05891,1.05986,1.05847,1.05958,11677.7197,-1.0,0.0,0.0
2,1.05957,1.06041,1.05744,1.05765,15785.2197,1.0,0.0,0.0
3,1.05765,1.05956,1.05708,1.05935,14816.4902,0.0,0.0,0.0
4,1.05934,1.05946,1.05835,1.05927,10641.8301,1.0,0.0,0.0
5,1.05925,1.06104,1.05890,1.06081,13180.9697,0.0,-1.0,-1.0
6,1.06082,1.06181,1.06036,1.06089,14419.7803,1.0,-1.0,-1.0
7,1.06092,1.06179,1.06055,1.06139,15307.1904,-1.0,-1.0,-1.0
8,1.06136,1.06153,1.05658,1.05724,25968.0293,-1.0,-1.0,-1.0
9,1.05719,1.05900,1.05668,1.05668,19795.0098,1.0,-1.0,-1.0


In [428]:
NumCLASS =  DATASET['Predicted_Class'].value_counts().shape[0]
print(NumCLASS)

3


In [429]:
class limits(SignalStrategy, TrailingStrategy):
    buy_lim = 0.99
    sell_lim = -0.99
    
    def init(self):
        super().init()
        self.Op_Abierta = False
        
    def next(self):
        sig = self.data.Position[-1]
        if not self.Op_Abierta and sig >= self.buy_lim:
            self.buy()
            self.Op_Abierta = True
        if self.Op_Abierta and sig <= self.sell_lim:
            self.sell()
            self.Op_Abierta = False

In [430]:
class limits2(SignalStrategy, TrailingStrategy):
    buy_lim = 0.99
    sell_lim = -0.99
    
    def init(self):
        super().init()
        
    def next(self):
        sig = self.data.Position[-1]
        if  sig >= self.buy_lim:
            self.buy()
        if  sig <= self.sell_lim:
            self.sell()

In [431]:
class Position(SignalStrategy):
    def init(self):
        global df
        super().init()  
        self.set_signal(self.data.Position)

In [432]:
DATASET['Predicted_Class'].value_counts()

 1.0    9745
-1.0    6088
 0.0    4471
Name: Predicted_Class, dtype: int64

In [433]:
DATASET['Real_Class'].value_counts()

 0.0    8146
 1.0    6085
-1.0    6073
Name: Real_Class, dtype: int64

In [434]:
STRATEGY_NAME = limits
print(STRATEGY_NAME.__name__)

limits


In [435]:
bt = Backtest(DATASET, STRATEGY_NAME, cash=CASH, commission=COMMISSION)#trade_on_close=False
bt.run()

Start                               0
End                             20303
Duration                        20303
Exposure [%]                  99.9015
Equity Final [$]              793.146
Equity Peak [$]               1033.62
Return [%]                   -20.6854
Buy & Hold Return [%]         4.97299
Max. Drawdown [%]            -26.4657
Avg. Drawdown [%]            -1.60845
Max. Drawdown Duration          19322
Avg. Drawdown Duration        965.286
# Trades                         4127
Win Rate [%]                  48.7764
Best Trade [%]                 1.6327
Worst Trade [%]              -1.45078
Avg. Trade [%]            -0.00546999
Max. Trade Duration                53
Avg. Trade Duration           4.91471
Expectancy [%]               0.121301
SQN                          -1.80076
Sharpe Ratio               -0.0284486
Sortino Ratio              -0.0358756
Calmar Ratio             -0.000206682
_strategy                      limits
dtype: object

In [436]:
bt.plot()

In [437]:
data = {'Model_Name':[MODEL_NAME], 'PRED':[PRED], 'Strategy_Class':[STRATEGY_NAME.__name__], 'Equity_Final_$':[bt._results['Equity Final [$]']],
       'Equity_Peak_$':[bt._results['Equity Peak [$]']],'Return_%':[bt._results['Return [%]']],'#_Trades':[bt._results['# Trades']],
       'Win_Rate':[bt._results['Win Rate [%]']],'Commission':[COMMISSION],'Cash':[CASH]}
Results = pd.DataFrame(data)
print(Results)

                            Model_Name  PRED Strategy_Class  Equity_Final_$  \
0  3C-1H-LSTM_1H_43F_Class_01_Test.csv  True         limits      793.145792   

   Equity_Peak_$   Return_%  #_Trades   Win_Rate  Commission  Cash  
0    1033.615103 -20.685421    4127.0  48.776351      0.0001  1000  


In [438]:
filename = 'BackTesting_Results.csv'
print(filename)
pwd = os.getcwd()
datasource_path = os.path.join(pwd, r'../../../Results/Backtesting/')
filename_path = os.path.join(pwd, r'../../../Results/Backtesting/' + filename)
print(filename_path)

BackTesting_Results.csv
C:\Users\marami3\Documents\ITESO\IDI4\forex_prediction\Jupyter Notebooks\IDI4\backtesting\../../../Results/Backtesting/BackTesting_Results.csv


In [439]:
#LOAD RESULTS .CSV
pwd = os.getcwd()
filenames = glob.glob(datasource_path + "/*.csv")
filenames.sort()
for file in filenames:
    print(os.path.basename(file))

BackTesting_Results.csv


In [440]:
Results_loaded = pd.read_csv(filenames[0], header=0)
print(Results_loaded)

                             Model_Name   PRED Strategy_Class  Equity_Final_$  \
0  3C-1H-LSTM_1H_43F_Class_01_Train.csv  False         limits    2.929920e+07   
1  3C-1H-LSTM_1H_43F_Class_01_Train.csv  False         limits    2.929920e+07   
2  3C-1H-LSTM_1H_43F_Class_01_Train.csv   True         limits    1.283255e+04   
3  3C-1H-LSTM_1H_43F_Class_01_Train.csv   True         limits    1.283255e+04   
4   3C-1H-LSTM_1H_43F_Class_01_Test.csv  False         limits    2.294242e+03   

   Equity_Peak_$      Return_%  #_Trades   Win_Rate  Commission  Cash  
0   2.948331e+07  2.929820e+06   18630.0  69.017713      0.0001  1000  
1   2.948331e+07  2.929820e+06   18630.0  69.017713      0.0001  1000  
2   1.291319e+04  1.183255e+03   11964.0  54.321297      0.0001  1000  
3   1.291319e+04  1.183255e+03   11964.0  54.321297      0.0001  1000  
4   2.325174e+03  1.294242e+02    5261.0  58.772097      0.0001  1000  


In [441]:
#print(Results)
Results_loaded = Results_loaded.append(Results,ignore_index=True)
print(Results_loaded)

                             Model_Name   PRED Strategy_Class  Equity_Final_$  \
0  3C-1H-LSTM_1H_43F_Class_01_Train.csv  False         limits    2.929920e+07   
1  3C-1H-LSTM_1H_43F_Class_01_Train.csv  False         limits    2.929920e+07   
2  3C-1H-LSTM_1H_43F_Class_01_Train.csv   True         limits    1.283255e+04   
3  3C-1H-LSTM_1H_43F_Class_01_Train.csv   True         limits    1.283255e+04   
4   3C-1H-LSTM_1H_43F_Class_01_Test.csv  False         limits    2.294242e+03   
5   3C-1H-LSTM_1H_43F_Class_01_Test.csv   True         limits    7.931458e+02   

   Equity_Peak_$      Return_%  #_Trades   Win_Rate  Commission  Cash  
0   2.948331e+07  2.929820e+06   18630.0  69.017713      0.0001  1000  
1   2.948331e+07  2.929820e+06   18630.0  69.017713      0.0001  1000  
2   1.291319e+04  1.183255e+03   11964.0  54.321297      0.0001  1000  
3   1.291319e+04  1.183255e+03   11964.0  54.321297      0.0001  1000  
4   2.325174e+03  1.294242e+02    5261.0  58.772097      0.0001  1000  


In [442]:
Results_loaded.to_csv(filename_path, header=True, index=False)